In [ ]:
# =====================================================
# FAKE NEWS DETECTION – (MiniLM version) + WELFAKE
# =====================================================

# 1. CÀI ĐẶT & IMPORT
!pip install -q transformers datasets torch scikit-learn pandas numpy psutil accelerate

import os, re, time, shutil, psutil, warnings
import pandas as pd
import numpy as np
import torch
from datasets import load_dataset, Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
from transformers import (
    BertTokenizerFast,
    BertForSequenceClassification,
    Trainer, TrainingArguments, EarlyStoppingCallback, set_seed,
    DataCollatorWithPadding
)
from google.colab import drive
from tqdm.auto import tqdm

warnings.filterwarnings("ignore")

# 2. KIỂM TRA GPU
if torch.cuda.is_available():
    device_name = torch.cuda.get_device_name(0)
    vram_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
else:
    device_name = "CPU"
    vram_gb = 0.0
print(f"Device: {device_name} | CUDA: {torch.cuda.is_available()} | VRAM: {vram_gb:.1f} GB")

# 3. MOUNT DRIVE & CHECK SPACE
drive.mount('/content/drive', force_remount=False)

def check_drive_space(path="/content/drive/MyDrive"):
    try:
        usage = psutil.disk_usage(path)
        used_gb = usage.used / 1e9
        total_gb = usage.total / 1e9
        pct = used_gb/total_gb*100 if total_gb>0 else 0
        print(f"Drive: {used_gb:.1f}GB / {total_gb:.1f}GB ({pct:.1f}%)")
        if pct > 90:
            print("CẢNH BÁO: Dung lượng Drive gần đầy!")
    except Exception as e:
        print("Không thể kiểm tra dung lượng Drive:", e)

check_drive_space()

# 4. OUTPUT_DIR + CHECKPOINT MANAGEMENT
OUTPUT_DIR = "/content/drive/MyDrive/WELFake_MiniLM_Pro"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def manage_checkpoints(output_dir, keep_latest=2):
    ckpts = [c for c in os.listdir(output_dir) if c.startswith("checkpoint-")]
    if len(ckpts) <= keep_latest:
        return

    def idx_from_name(n):
        parts = n.split("-")
        try:
            return int(parts[-1])
        except:
            return os.path.getmtime(os.path.join(output_dir, n))

    ckpts_sorted = sorted(ckpts, key=idx_from_name)
    for old_ckpt in ckpts_sorted[:-keep_latest]:
        shutil.rmtree(os.path.join(output_dir, old_ckpt), ignore_errors=True)
        print(f"Đã xóa checkpoint cũ: {old_ckpt}")

def get_last_checkpoint(output_dir):
    ckpts = [c for c in os.listdir(output_dir) if c.startswith("checkpoint-")]
    if not ckpts:
        return None

    def idx_from_name(n):
        parts = n.split("-")
        try:
            return int(parts[-1])
        except:
            return os.path.getmtime(os.path.join(output_dir, n))

    ckpts_sorted = sorted(ckpts, key=idx_from_name, reverse=True)
    manage_checkpoints(output_dir, keep_latest=2)
    return os.path.join(output_dir, ckpts_sorted[0])

last_checkpoint = get_last_checkpoint(OUTPUT_DIR)
print("Checkpoint gần nhất:", last_checkpoint or "Không có → Train từ đầu")

# 5. TẢI & TIỀN XỬ LÝ DỮ LIỆU
print("\nTải dataset WELFake...")
dataset = load_dataset("davanstrien/WELFake")
df = pd.DataFrame(dataset['train'])
print("Số mẫu ban đầu:", len(df))

df['content'] = df.get('title', '').fillna('') + " [SEP] " + df.get('text', '').fillna('')

def clean_text(text):
    if not isinstance(text, str): return ""
    t = text.lower()
    t = re.sub(r'https?://\S+|www\.\S+', ' ', t)
    t = re.sub(r'<.*?>', ' ', t)
    t = re.sub(r'[^a-zA-Z0-9\s]', ' ', t)
    t = re.sub(r'\s+', ' ', t).strip()
    return t

print("Đang làm sạch văn bản...", end="")
df['content'] = df['content'].apply(clean_text)
df = df[df['content'].str.len() > 20].drop_duplicates(subset=['content'])
print(f" → Sau xử lý: {len(df):,}")

label_counts = df['label'].value_counts(normalize=True)
print(f"Phân bố nhãn: Fake={label_counts.get(0,0):.1%}, Real={label_counts.get(1,0):.1%}")

classes = np.array([0,1])
class_weights = compute_class_weight('balanced', classes=classes, y=df['label'])
class_weight_dict = {0: float(class_weights[0]), 1: float(class_weights[1])}
print("Class weights:", class_weight_dict)

# 6. SPLIT
labels = df['label'].values
train_df, temp_df = train_test_split(df, test_size=0.25, random_state=42, stratify=labels)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['label'])

print(f"Train: {len(train_df):,} | Val: {len(val_df):,} | Test: {len(test_df):,}")

train_dataset = Dataset.from_pandas(train_df[['content','label']].reset_index(drop=True))
val_dataset   = Dataset.from_pandas(val_df[['content','label']].reset_index(drop=True))
test_dataset  = Dataset.from_pandas(test_df[['content','label']].reset_index(drop=True))
dataset_dict = DatasetDict({"train": train_dataset, "validation": val_dataset, "test": test_dataset})

# 7. TOKENIZER MiniLM
tokenizer = BertTokenizerFast.from_pretrained("microsoft/MiniLM-L6-H384-uncased")

def tokenize_fn(batch):
    return tokenizer(batch["content"], truncation=True, max_length=384, padding=False)

print("Tokenizing dataset...")
tokenized = dataset_dict.map(tokenize_fn, batched=True, batch_size=1000, remove_columns=['content'])
tokenized = tokenized.rename_column("label", "labels")

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 8. LOAD MiniLM MODEL
model = BertForSequenceClassification.from_pretrained(
    "microsoft/MiniLM-L6-H384-uncased",
    num_labels=2
)

model.config.id2label = {0:"Fake", 1:"Real"}
model.config.label2id = {"Fake":0, "Real":1}

# 9. TRAINING ARGUMENTS
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,
    warmup_ratio=0.1,
    weight_decay=0.01,
    learning_rate=3e-5,
    lr_scheduler_type="linear",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=100,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    fp16=torch.cuda.is_available(),
    dataloader_num_workers=2,
    report_to="none",
)

# 10. METRICS
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted", zero_division=0)
    acc = accuracy_score(labels, preds)
    try:
        probs = torch.softmax(torch.tensor(logits), dim=1).numpy()[:,1]
        auc = roc_auc_score(labels, probs)
    except:
        auc = None
    return {"accuracy": float(acc), "precision": float(precision), "recall": float(recall), "f1": float(f1), "auc": auc}

# 11. WEIGHTED TRAINER
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        weight_tensor = torch.tensor([class_weights[0], class_weights[1]], dtype=torch.float32, device=model.device)
        loss_fct = torch.nn.CrossEntropyLoss(weight=weight_tensor)
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

# 12. TRAINER
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
)

# 13. TRAIN
print("\n" + "="*60)
print("BẮT ĐẦU HUẤN LUYỆN – MINI-LM")
print("="*60)

trainer.train(resume_from_checkpoint=last_checkpoint)

manage_checkpoints(OUTPUT_DIR, keep_latest=2)

# 14. TEST
print("\nĐÁNH GIÁ TRÊN TẬP TEST")
results = trainer.evaluate(tokenized["test"])
for k, v in results.items():
    print(f"{k}: {v}")

# 15. SAVE MODEL
final_model_path = os.path.join(OUTPUT_DIR, "final_best_model")
trainer.save_model(final_model_path)
tokenizer.save_pretrained(final_model_path)

print(f"\nĐÃ LƯU MÔ HÌNH TỐT NHẤT TẠI:\n{final_model_path}")

# Cleanup
for f in os.listdir(OUTPUT_DIR):
    if f.startswith("checkpoint-") or f == "runs":
        shutil.rmtree(os.path.join(OUTPUT_DIR, f), ignore_errors=True)

print("Dọn dẹp hoàn tất. Chỉ giữ final model!")


Device: CPU | CUDA: False | VRAM: 0.0 GB


ValueError: mount failed